In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split # 학습, 테스트set 구분
from sklearn.tree import export_graphviz # tree 시각화를 위해
import graphviz # tree 시각화
from sklearn.metrics import f1_score# 성능지표를 계산하기 위해 import
from sklearn.model_selection import cross_val_score, cross_validate # 교차검증

In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/LGaimers/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/LGaimers/test.csv')

In [4]:
print(train_df.shape)
print(test_df.shape)

(598, 2881)
(310, 2879)


## 사용 함수 정리

In [5]:
# 모두 결측치인 변수 제거
def remove_all_nan(dataframe):
  col_list = dataframe.columns
  nan_list = []
  nan_cnt = []
  nan_col = []
  full_list = []

  for col in col_list:
    if dataframe[col].isnull().sum() == 0:
      full_list.append(col)
      continue
    nan_list.append([col,dataframe[col].isnull().sum()])
    nan_cnt.append(dataframe[col].isnull().sum())
    nan_col.append(col)

  del_col = []
  for nan in nan_list:
    if nan[1] == len(dataframe):
      del_col.append(nan[0])
  
  return dataframe.drop(columns=del_col)

In [6]:
'''값이 1개 존재하면 제거'''

def remove_one_value(dataframe):
  del_col = []
  col_list = dataframe.columns
  for col in col_list:
      if dataframe[col].nunique()==1 :
          del_col.append(col)
  
  return dataframe.drop(columns=del_col)

## PRODUCT_CODE 별 split

In [7]:
trainA_31 = train_df[train_df['PRODUCT_CODE'] == 'A_31']
trainT_31 = train_df[train_df['PRODUCT_CODE'] == 'T_31']
trainO_31 = train_df[train_df['PRODUCT_CODE'] == 'O_31']

In [8]:
testA_31 = test_df[test_df['PRODUCT_CODE'] == 'A_31']
testT_31 = test_df[test_df['PRODUCT_CODE'] == 'T_31']
testO_31 = test_df[test_df['PRODUCT_CODE'] == 'O_31']

In [9]:
trainA_31 = remove_all_nan(trainA_31)
trainT_31 = remove_all_nan(trainT_31)
trainO_31 = remove_all_nan(trainO_31)

In [10]:
trainA_31 = remove_one_value(trainA_31)
trainT_31 = remove_one_value(trainT_31)
trainO_31 = remove_one_value(trainO_31)

## raw data 준비

x_train
- trainA_31_x
- trainT_31_x
- trainO_31_x

y_train
- trainA_31_y_r
- trainT_31_y_r
- trainO_31_y_r  

- trainA_31_y_c
- trainT_31_y_c
- trainO_31_y_c

test
- testA_31_x
- testT_31_x
- testO_31_x

In [32]:
trainA_31[trainA_31['Y_Class']==2]

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,X_128,X_129,X_130,X_131,X_132,...,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,NaN,NaN,19854.0,19854.0,0.20,...,193.296552,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,NaN,NaN,19856.0,19856.0,0.20,...,181.920690,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78
5,TRAIN_005,2,0.537833,2022-06-13 5:55,T050307,NaN,NaN,19859.0,19859.0,0.20,...,190.310345,184.7,384.0,368.622641,352.0,38.16,51.25,56.01,37.41,71.37
7,TRAIN_007,2,0.540003,2022-06-13 6:11,T050307,NaN,NaN,19861.0,19861.0,0.20,...,184.724138,173.8,393.0,370.283019,353.0,39.00,35.95,59.51,30.49,72.77
9,TRAIN_009,2,0.538049,2022-06-13 6:28,T050307,NaN,NaN,19864.0,19864.0,0.20,...,180.290000,167.2,393.0,372.870370,353.0,34.40,35.88,50.36,38.45,71.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,TRAIN_556,2,0.560568,2022-08-28 2:54,T050304,837.0,837.0,NaN,NaN,0.19,...,201.944828,194.6,384.0,368.433962,353.0,53.49,58.76,42.22,57.92,66.18
557,TRAIN_557,2,0.559463,2022-08-28 3:03,T050304,851.0,851.0,NaN,NaN,0.19,...,160.517241,145.6,393.0,376.698113,362.0,50.87,57.13,53.07,56.21,67.22
562,TRAIN_562,2,0.541429,2022-09-03 1:14,T050304,8164.0,8164.0,NaN,NaN,0.19,...,170.831034,160.2,383.0,368.660377,353.0,56.38,57.10,51.49,56.08,64.40
563,TRAIN_563,2,0.537465,2022-09-03 1:22,T050304,8185.0,8185.0,NaN,NaN,0.19,...,181.868966,171.6,383.0,369.018868,353.0,49.92,54.66,50.56,48.63,62.17


In [12]:
trainA_31_x = trainA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','LINE'])
trainT_31_x = trainT_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','LINE'])
trainO_31_x = trainO_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','LINE'])

In [13]:
trainA_31_x = trainA_31_x.fillna(-1)
trainT_31_x = trainT_31_x.fillna(-1)
trainO_31_x = trainO_31_x.fillna(-1)

In [14]:
testA_31_x = testA_31[trainA_31_x.columns]
testT_31_x = testT_31[trainT_31_x.columns]
testO_31_x = testO_31[trainO_31_x.columns]

In [15]:
testA_31_x = testA_31_x.fillna(-1)
testT_31_x = testT_31_x.fillna(-1)
testO_31_x = testO_31_x.fillna(-1)

In [16]:
# GCN에 넣기 위해 dataframe -> tensor 변환
trainA_31_x = torch.from_numpy(trainA_31_x.values)
trainT_31_x = torch.from_numpy(trainT_31_x.values)
trainO_31_x = torch.from_numpy(trainO_31_x.values)

testA_31_x = torch.from_numpy(testA_31_x.values)
testT_31_x = torch.from_numpy(testT_31_x.values)
testO_31_x = torch.from_numpy(testO_31_x.values)

In [17]:
# y_class 표현
trainA_31_y_c = pd.get_dummies(trainA_31['Y_Class'].astype('str'))
trainT_31_y_c = pd.get_dummies(trainT_31['Y_Class'].astype('str'))
trainO_31_y_c = pd.get_dummies(trainO_31['Y_Class'].astype('str'))

In [18]:
# classification
trainA_31_y_c = torch.from_numpy(trainA_31_y_c.values)
trainT_31_y_c = torch.from_numpy(trainT_31_y_c.values)
trainO_31_y_c = torch.from_numpy(trainO_31_y_c.values)

# regression
trainA_31_y_r = torch.from_numpy(trainA_31['Y_Quality'].values)
trainT_31_y_r = torch.from_numpy(trainT_31['Y_Quality'].values)
trainO_31_y_r = torch.from_numpy(trainO_31['Y_Quality'].values)

## Adj 구축
- 같은 line끼리 edge 구축

In [19]:
line_A = trainA_31['LINE']
line_T = trainT_31['LINE']
line_O = trainO_31['LINE']

In [20]:
adj_A = np.array([[1 if i==j else 0 for i in line_A] for j in line_A])
adj_T = np.array([[1 if i==j else 0 for i in line_T] for j in line_T])
adj_O = np.array([[1 if i==j else 0 for i in line_O] for j in line_O])

In [21]:
print(adj_A.shape, adj_T.shape, adj_O.shape)

(249, 249) (343, 343) (6, 6)


## Model 구축

In [22]:
import numpy as np

def crossEntropyError(y, t):
    delta = 1e-7 
    return -np.sum(t*np.log(y+delta))

t = np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0]) # label = 5
y = np.array([0.1, 0.03, 0.05, 0.2, 0.9, 0.0, 0.1, 0.2, 0.12, 0.03])

print("-- 정답인 경우 --")
print("CEE :", crossEntropyError(y, t))

y = np.array([0.1, 0.03, 0.05, 0.2, 0.0, 0.1, 0.2, 0.12, 0.03, 0.9])
print("-- 오류인 경우 --")
print("CEE :", crossEntropyError(y, t))

-- 정답인 경우 --
CEE : 0.1053604045467214
-- 오류인 경우 --
CEE : 16.118095650958317


In [33]:
import torch.nn.functional as F

class WeightedFocalLoss(nn.Module):
    def __init__(self, alpha=.25, gamma=2):
        super(WeightedFocalLoss, self).__init__()
        self.alpha = torch.tensor([alpha, 1-alpha]).cuda()
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        targets = targets.type(torch.long)
        at = self.alpha.gather(0, targets.data.view(-1))
        pt = torch.exp(-BCE_loss)
        F_loss = at*(1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

In [224]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class Loss(nn.Module):
    def __init__(self):
        super(Loss, self).__init__()
        weights = torch.FloatTensor([0.3,0.33,0.3])
        self.loss = nn.CrossEntropyLoss(reduction='mean', weight=weights)
    def forward(self, output, labels):
        labels = torch.argmax(labels, dim=1)
        loss = self.loss(output, labels)
        # print('---------')
        # print(output.shape, labels.shape)
        # print(output,labels)
        return torch.mean(loss)

def sample_mask(idx, l):
    mask = np.zeros(l)
    mask[idx] = 1
#    return np.array(mask, dtype=np.bool)
    return np.array(mask, dtype=bool)


def build_optimizer(model, lr, weight_decay):
    gcn1, gcn2 = [], []
    for name, p in model.named_parameters():
        if 'layer1' in name:
            gcn1.append(p)
        else:
            gcn2.append(p)
    opt = optim.Adam([{'params': gcn1, 'weight_decay': weight_decay},
                      {'params': gcn2}
                      ], lr=lr)
    return opt


def get_lr():
    pass


def get_loss(output, labels):
    loss = Loss()
    return loss(output, labels)


def get_accuracy(outputs, labels):
    outputs = torch.argmax(outputs, dim=1)
    labels = torch.argmax(labels, dim=1)
    outputs = outputs.numpy()
    labels = labels.numpy()
    correct = outputs == labels
    tp = np.sum(correct)
    return tp

In [235]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as Func


def preprocess_adj(A):
    '''
    Pre-process adjacency matrix
    :param A: adjacency matrix
    :return:
    '''
    I = np.eye(A.shape[0])
    A_hat = A + I # add self-loops
    D_hat_diag = np.sum(A_hat, axis=1)
    D_hat_diag_inv_sqrt = np.power(D_hat_diag, -0.5)
    D_hat_diag_inv_sqrt[np.isinf(D_hat_diag_inv_sqrt)] = 0.
    D_hat_inv_sqrt = np.diag(D_hat_diag_inv_sqrt)
    return np.dot(np.dot(D_hat_inv_sqrt, A_hat), D_hat_inv_sqrt)


class GCNLayer(nn.Module):
    def __init__(self, in_dim, out_dim, acti=True):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_dim, out_dim) # bias = False is also ok.
        if acti:
            # self.acti = nn.Tanh()
            self.acti = nn.ReLU(inplace=True)
            # self.acti = nn.LeakyReLU(inplace=True)
            # self.acti = nn.ELU(inplace=True)
            #self.acti = nn.GELU()
            
        else:
            self.acti = None
    def forward(self, F):
        output = self.linear(F)
        if not self.acti:
            return output
        return self.acti(output)


class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, p):
        super(GCN, self).__init__()
        # self.gcn_layer1 = GCNLayer(input_dim, hidden_dim, acti=False)
        self.gcn_layer1 = GCNLayer(input_dim, input_dim, acti=False)
        self.gcn_layer2 = GCNLayer(input_dim, hidden_dim, acti=False)
        self.gcn_layer3 = GCNLayer(input_dim, hidden_dim)
        self.gcn_layer4 = GCNLayer(hidden_dim, num_classes, acti=False)
        self.dropout = nn.Dropout(p)

    def forward(self, A, X):
        A = torch.from_numpy(preprocess_adj(A)).float()
        X = self.dropout(X.float())
        F = torch.mm(A, X)
        F = self.gcn_layer1(F)
        F = self.dropout(F)
        F = torch.mm(A, F)
        F = self.gcn_layer2(F)
        F = self.dropout(F)
        F = torch.mm(A, F)
        # F = self.gcn_layer3(F)
        # F = self.dropout(F)
        # F = torch.mm(A, F)
        output = self.gcn_layer4(F)
        return output

In [244]:
# import argparse

# parser = argparse.ArgumentParser()
# # parser.add_argument('--dataset', type=str, default='citeseer', help='Dataset to train')
# parser.add_argument('--init_lr', type=float, default=0.01, help='Initial learing rate')
# parser.add_argument('--epoches', type=int, default=200, help='Number of traing epoches')
# parser.add_argument('--hidden_dim', type=list, default=16, help='Dimensions of hidden layers')
# parser.add_argument('--dropout', type=float, default=0.5, help='Dropout rate (1 - keep  probability)')
# parser.add_argument('--weight_decay', type=float, default=5e-4, help='Weight for l2 loss on embedding matrix')
# parser.add_argument('--log_interval', type=int, default=10, help='Print iterval')
# parser.add_argument('--log_dir', type=str, default='experiments', help='Train/val loss and accuracy logs')
# parser.add_argument('--checkpoint_interval', type=int, default=20, help='Checkpoint saved interval')
# parser.add_argument('--checkpoint_dir', type=str, default='checkpoints', help='Directory to save checkpoints')
# args = parser.parse_args()

import easydict

args = easydict.EasyDict({
    "init_lr": 0.01,
    "epoches": 1000,
    "hidden_dim": 16,
    'dropout': 0.3,
    'weight_decay':1e-4,
    'log_interval':10,
    'checkpoint_interval':20
})

## Train

In [245]:
import os
import torch
import torch.nn.functional as F

def train(model,optimizer,adj,features,y_train):
        
    for epoch in range(args.epoches + 1):
        outputs = model(adj, features)
        loss = get_loss(outputs, y_train)
        
        model.eval()
        outputs = model(adj, features)
        train_accuracy = get_accuracy(outputs, y_train)
        
        model.train()
        if epoch % args.log_interval == 0:
            print("Epoch: %d, train loss: %f, train ac: %f"
                  %(epoch, loss.detach().numpy(), train_accuracy))
        if epoch % args.checkpoint_interval == 0:
            torch.save(model.state_dict(), os.path.join('/content/drive/MyDrive/LGaimers/model/', "gcn_%d.pth"%epoch))
        optimizer.zero_grad()  # Important
        loss.backward()
        optimizer.step()

    print(outputs)

    return F.softmax(outputs,dim=1)

In [246]:
model_A = GCN(trainA_31_x.shape[1], args.hidden_dim, 3, args.dropout)
model_T = GCN(trainT_31_x.shape[1], args.hidden_dim, 3, args.dropout)
model_O = GCN(trainO_31_x.shape[1], args.hidden_dim, 3, args.dropout)

optimizer_A = build_optimizer(model_A, args.init_lr, args.weight_decay)
optimizer_T = build_optimizer(model_T, args.init_lr, args.weight_decay)
optimizer_O = build_optimizer(model_O, args.init_lr, args.weight_decay)

In [247]:
train(model_A,optimizer_A,adj_A,trainA_31_x,trainA_31_y_c)

Epoch: 0, train loss: 10859.167969, train ac: 118.000000
Epoch: 10, train loss: 245484.437500, train ac: 118.000000
Epoch: 20, train loss: 95199.546875, train ac: 105.000000
Epoch: 30, train loss: 90049.382812, train ac: 71.000000
Epoch: 40, train loss: 73158.531250, train ac: 118.000000
Epoch: 50, train loss: 8783.863281, train ac: 118.000000
Epoch: 60, train loss: 10292.660156, train ac: 120.000000
Epoch: 70, train loss: 22160.595703, train ac: 113.000000
Epoch: 80, train loss: 9424.424805, train ac: 91.000000
Epoch: 90, train loss: 12257.263672, train ac: 71.000000
Epoch: 100, train loss: 9710.085938, train ac: 118.000000
Epoch: 110, train loss: 3716.904053, train ac: 131.000000
Epoch: 120, train loss: 4354.526367, train ac: 100.000000
Epoch: 130, train loss: 3240.772705, train ac: 102.000000
Epoch: 140, train loss: 8305.749023, train ac: 113.000000
Epoch: 150, train loss: 4067.965576, train ac: 118.000000
Epoch: 160, train loss: 1728.368652, train ac: 113.000000
Epoch: 170, train l

tensor([[2.5440e-12, 1.0000e+00, 0.0000e+00],
        [2.7853e-25, 1.0000e+00, 0.0000e+00],
        [2.5440e-12, 1.0000e+00, 0.0000e+00],
        [2.7853e-25, 1.0000e+00, 0.0000e+00],
        [2.5440e-12, 1.0000e+00, 0.0000e+00],
        [2.7853e-25, 1.0000e+00, 0.0000e+00],
        [2.5440e-12, 1.0000e+00, 0.0000e+00],
        [2.7846e-25, 1.0000e+00, 0.0000e+00],
        [2.5440e-12, 1.0000e+00, 0.0000e+00],
        [2.7832e-25, 1.0000e+00, 0.0000e+00],
        [2.5440e-12, 1.0000e+00, 0.0000e+00],
        [2.7866e-25, 1.0000e+00, 0.0000e+00],
        [2.5440e-12, 1.0000e+00, 0.0000e+00],
        [2.7839e-25, 1.0000e+00, 0.0000e+00],
        [2.5440e-12, 1.0000e+00, 0.0000e+00],
        [2.7846e-25, 1.0000e+00, 0.0000e+00],
        [2.5440e-12, 1.0000e+00, 0.0000e+00],
        [2.7846e-25, 1.0000e+00, 0.0000e+00],
        [2.5391e-12, 1.0000e+00, 0.0000e+00],
        [2.7853e-25, 1.0000e+00, 0.0000e+00],
        [2.5440e-12, 1.0000e+00, 0.0000e+00],
        [2.5440e-12, 1.0000e+00, 0